In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions  import *

from dotenv import load_dotenv
import sys
import os
import datetime
# Добавляем корневую директорию проекта в sys.path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

from consultant.tools.basic import normalize_path

load_dotenv()

spark = (SparkSession.builder
         .appName("cbr_currency_etl")
         .getOrCreate())

spark

KeyboardInterrupt: 

In [ ]:
from consultant.extractors.base import get_cbr_currency_rate_json


currency_list = list(get_cbr_currency_rate_json()['Valute'].values())

raw_currency = spark.createDataFrame(currency_list)
raw_currency.show(100, truncate=False)

/home/b.kustov/.local/lib/python3.8/site-packages/pyspark/sql/session.py:381: UserWarning: inferring schema from dict is deprecated,please use pyspark.sql.Row instead
  warnings.warn("inferring schema from dict is deprecated,"


+--------+-------+----------------------------------------+-------+-------+--------+--------+
|CharCode|ID     |Name                                    |Nominal|NumCode|Previous|Value   |
+--------+-------+----------------------------------------+-------+-------+--------+--------+
|AUD     |R01010 |Австралийский доллар                    |1      |036    |65.8247 |66.7047 |
|AZN     |R01020A|Азербайджанский манат                   |1      |944    |60.7475 |61.1471 |
|GBP     |R01035 |Фунт стерлингов Соединенного королевства|1      |826    |131.5462|132.4739|
|AMD     |R01060 |Армянских драмов                        |100    |051    |26.1876 |26.3625 |
|BYN     |R01090B|Белорусский рубль                       |1      |933    |29.885  |29.8278 |
|BGN     |R01100 |Болгарский лев                          |1      |975    |55.584  |55.8436 |
|BRL     |R01115 |Бразильский реал                        |1      |986    |17.0622 |17.2339 |
|HUF     |R01135 |Венгерских форинтов                     |1

In [13]:
prepared_currency = raw_currency \
    .withColumn('VunitRate', col('Value') / col('Nominal')) \
    .withColumn('ValCursDate', datetime.date.today()) \
    .select(['CharCode', 'Nominal', 'Value', 'VunitRate', 'ValCursDate'])
prepared_currency.show(100, truncate=False)

AssertionError: col should be Column

In [14]:
spark.stop()